# Eternal Card Extractor

## Section A: Make sure you have the required python 3 modules
This script was built on Python 3.x and uses pyautogui and opencv 3.4.2. I think the rest of the default Python 3 modules are safe. If the cells below throw errors, you may have problems.

In [32]:
import sys
assert sys.version[0]=='3'

In [35]:
import cv2
assert cv2.__version__=='3.4.2'
    # cv2 versions above 3.4.2 do not seem to have xfeatures2d.SIFT methods directly available.
    # When in doubt, force the installation of version 3.4.2.16. I used
    #   pip3 install --user opencv-python==3.4.2.16
    #   pip3 install --user opencv-contrib-python==3.4.2.16
    # Both opencv-python and open-contrib-python seemed to be needed for SIFT to work.

In [33]:
import pyautogui
import pickle
import gzip

## Section B: Grab screenshots of your card collection
For this section, get Eternal running (maximized is easier) and head to the first screen of your card collection. Afterwards Alt-Tab back to this Jupyter notebook, make sure it is unmaximized and in a position that leaves Eternal visible on the top left corner (so that Python can simulate a click on the Eternal client and bring it to the foreground to take the screenshots), and run the cells below.

(DISCLAIMER: I did not actually run the Section B code from this Jupyter notebook - I ran it from the command line using a script because I do not have Jupyter on the machine where I have Eternal installed. The script I did use is also available on the github repo. Be careful in any case: the script for reasons unknown generates two screenshots of the first page of the collection, which is ok but meeses up the number of cards owned of those cards. If you end up with duplicate screenshots, clean up!)

In [12]:
import pyautogui

In [13]:
max_screenshots=150
eternal_x,eternal_y=10,10
    # Position to click on in order to bring Eternal to the foreground
next_x,next_y=0.855*pyautogui.size()[0],0.525*pyautogui.size()[1]
    # Position to click to move to the next collection page 
    # next_x,next_y=1170,400 # Works well for a screen resolution of 1366 x 768
screenshots_folder='./Screenshots/'

In [ ]:
pyautogui.click(eternal_x,eternal_y)

cnt=0
old_screenshot=None
done=False
while (not done) and (cnt<max_screenshots):
    cnt=cnt+1
    new_screenshot=pyautogui.screenshot()
    if new_screenshot!=old_screenshot:
        new_screenshot.save(screenshots_folder+'ss{:03d}.png'.format(cnt))
        old_screenshot=new_screenshot
        pyautogui.click(next_x,next_y)
    else:
        done=True

## Section C: Identify card collection

### Section C.1: Load card artwork features

The library is based on the one in Eternal Warcry - thanks for allowing me to use it for this project!

The pickle module is very nice but can lead to incompatibilities. If it cannot unpickle the library, let me know so that I can think of an alternative way of serializing it.

In [1]:
import pickle
import gzip

In [2]:
library_local='./card_library.gz'
    # A pickled version of the library with the artwork descriptors already calculated
    # that I prepared so that you don't have to.

In [3]:
with gzip.open(library_local,'rb') as library_file:
    library=pickle.load(library_file)

### Section C.2: Process screenshots

In [4]:
import pyautogui
import cv2
import os

In [5]:
img_w,img_h=80,80
    # Size in pixels of the box used to extract features from.
    # Limited by either the resolution of your screen or that of the card artwork.
    # 80x80 was used to construct the card library.
nfeatures=50
    # Number of features to extract from each box. 50 works well with 80x80 boxes.

    # This area has some upside potential by the way - identifying less features based
    # on lower resolution boxes would speed up the process and produce a smaller
    # card library file. To be explored.
sift=cv2.xfeatures2d.SIFT_create(nfeatures=nfeatures)

The functions below seem fairly common for this specific purpose. My understanding of them is incomplete, but the first one seems to return the good matching features between two descriptor sets (and will be used once between the target and each of the possible option images), and the second one seems to compute how may of the total matching features that were considered good came from each classification option (and will be used once, after all of the options have been evaluated).

The option that generated the highest number of good matching features is the most likely one.

In [6]:
def knn_match(des1, des2, nn_ratio=0.7):
    index_params = dict(algorithm = 0, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < nn_ratio * n.distance:
            good.append(m)
    return good

def knn_clasif(good_matches):
    logprobs=[]
    best_template, highest_logprob = None, 0.0
    sum_good_matches = sum([len(gm) for gm in good_matches])
    for i, gm in enumerate(good_matches):
        logprob = len(gm)/sum_good_matches
        logprobs.append(logprob)
        if logprob > highest_logprob:
            highest_logprob = logprob
            best_template = i
    return best_template,logprobs

The parameters below define the places in the screenshots where we will extract image boxes and number owned diamonds from. Plenty of options would be available; I went for estimating:
- The distance between cards (sep_x,sep_y), which when aplied to the position of the top left corner of the first image box (crd_x,crd_y) generates the boxes that will be used to extract owned card features (pos_lst)
- The distance between the cards (the same sep_x,sep_y) and the horizontal distance between the ownership diamonds (sep_cnt), which when applied to the position of the center of the second diamond (the first one is a given - you always have at least one card of those present in the screenshots) of the top left card (cnt_x,cnt_y) generates the points to check if an ownership diamond is present (cnt_lst). This test is done against a cutoff of 100, but the actual values measured on the screenshots were 330 on yes-diamond and 15 on no-diamond

There is another notebook in the repo to help finetune these parameters.

In [7]:
screenshots_folder='./Screenshots/'
collection_file='./my_collection.txt'

sep_x,sep_y=int(0.122*pyautogui.size()[0]),int(0.383*pyautogui.size()[1])
    # Represents the distances between equivalent positions of cards on the screenshot,
    # i.e., 'top left to top left' or 'center to center', or whatever
    # sep_x,sep_y=166,295 # Works well for a screen resolution of 1366 x 768   
crd_x,crd_y=int(0.172*pyautogui.size()[0]),int(0.211*pyautogui.size()[1])
    # Represents the top left corner of the feature extraction box of the first card
    # crd_x,crd_y=234,162 # Works well for a screen resolution of 1366 x 768
pos_lst=[ (crd_x+n_x*sep_x,crd_y+n_y*sep_y,img_w,img_h)
          for n_y in range(2) for n_x in range(6)]
    # Represent the list of feature extraction boxes of the 12 cards

sep_cnt=int(0.013*pyautogui.size()[0])
    # Represents the horizontal distance between equivalent positions of diamods on the screenshot
    # i.e., 'top left to top left' or 'center to center', or whatever
    # sep_cnt=17 # Works well for a screen resolution of 1366 x 768
cnt_x,cnt_y=int(0.192*pyautogui.size()[0]),int(0.146*pyautogui.size()[1])
    # Represents the center of the second ownership diamond of the first card
    #cnt_x,cnt_y=262,112 # Works well for a screen resolution of 1366 x 768
cnt_lst=[ [(cnt_x+n_x*sep_x+n_cnt*sep_cnt,cnt_y+n_y*sep_y) for n_cnt in range(3)]
          for n_y in range(2) for n_x in range(6)]
    # Represents the center of the second, third, and fourth diamonds of the 12 cards

owned_threshold=100
    # Threshold to determine whether at the point there is a yes-diamond or no-diamond.
    # The actual numbers are 330 for yes-diamond and 15 for no-diamond

The script extracts each card in each screenshot of the collection, compares its features with those of all the cards in the library, and finds the best match.

It can be made much faster by identifying additional features from the cards, such as power requirements profile, cost, or attack; these can then be used to narrow the search for the target card, which would also allow to reduce the size of the feature extraction boxes and the number of features being extracted from each box.

I also tried text extraction, but tesseract did not work well on the card names, at least not on my low resolution screenshots. That however does not mean it cannot be used to speed up the identification process, as long as the extracted gibberish is consistent.

To be explored.

In [8]:
%%time

list_cards=list(library)
for card in library:
    library[card]['owned']=0
errors=[]
tot=len(os.listdir(screenshots_folder))
cnt=0
for ss_file in sorted(os.listdir(screenshots_folder)):
    cnt=cnt+1
    print('[{:3d}/{:3d}] Processing file {}...'.format(cnt,tot,ss_file))
    if ss_file[-3:]=='png':
        screenshot=cv2.imread(screenshots_folder+ss_file)
        for i in range(12):
            l,t,w,h=pos_lst[i]
            target=cv2.resize(screenshot[t:t+h,l:l+w],(img_w,img_h))
            try:
                _,descriptors=sift.detectAndCompute(target,None)
                list_good_matches=[]
                for card in library:
                    list_good_matches.append(knn_match(library[card]['descriptors'],descriptors))
                best_match,probs=knn_clasif(list_good_matches)
                num_owned=1+sum([ screenshot[c[1],c[0],:].sum()>owned_threshold for c in cnt_lst[i]])
                print(12*' '+'- {} '.format(num_owned)+list_cards[best_match])
                library[list_cards[best_match]]['owned']=min(4,library[list_cards[best_match]]['owned']+num_owned)
            except:
                errors.append('[ERROR] Problem identifying card {:2d} in screenshot {}!'.format(i+1,ss_file))
                print(errors[-1])
                pass

[  1/ 99] Processing file .ipynb_checkpoints...
[  2/ 99] Processing file grab_ss.py...
[  3/ 99] Processing file ss001.png...
            - 4 Seek Power
            - 1 Fire Sigil
            - 4 Granite Monument
            - 1 Granite Monument
            - 4 Granite Waystone
            - 3 Shugo Standard
            - 3 Helpful Doorbot
            - 1 Trail Stories
            - 4 Bore
            - 3 Drifter
            - 4 Firemane Cub
            - 4 Forge Wolf
[  4/ 99] Processing file ss002.png...
            - 4 Grenadin Drone
            - 4 Heavy Axe
            - 4 Hidden Shiv
            - 1 Hidden Shiv
            - 3 Iceberg Frontrunner
            - 1 Kaleb's Intervention
            - 4 Light 'em Up
            - 2 Mindfire
            - 4 On the Hunt
            - 4 Oni Dragonsmith
            - 4 Oni Ronin
            - 4 Pummel
[  5/ 99] Processing file ss003.png...
            - 4 Pyroknight
            - 4 Rambot
            - 4 Ruin
            - 1 Ruin
       

            - 4 Synchronized Strike
            - 4 Talir's Favored
            - 1 Talir's Favored
            - 4 Teleport
            - 4 Temple Scribe
            - 4 Trail Maker
            - 4 Trail Runner
            - 1 Voice of the Speaker
            - 4 Xenan Initiation
            - 4 Ageless Mentor
            - 4 Amber Acolyte
[ 24/ 99] Processing file ss022.png...
            - 4 Amber Ring
            - 4 Avirax Familiar
            - 3 Baying Serasaur
            - 2 Clockroach
            - 2 Dawnwalker
            - 4 Decay
            - 1 Decay
            - 2 Deft Strike
            - 4 Determined Stranger
            - 4 Dilphex Stalker
            - 4 Disciplined Amanera
            - 4 Dispel
[ 25/ 99] Processing file ss023.png...
            - 1 Dispel
            - 2 Gear Master
            - 2 Gravetender
            - 2 Initiation Bell
            - 2 Lastlight Infusion
            - 1 Lastlight Infusion
            - 2 Lunar Magus
            - 2 Moondial
 

            - 1 Citywide Ban
            - 2 Copperhall Blessing
            - 4 Copperhall Paladin
            - 4 Copperhall Recruit
            - 2 Copperhall Recruit
[ 43/ 99] Processing file ss041.png...
            - 4 Crownwatch Cavalry
            - 4 Crownwatch Press-Gang
            - 1 Frontier Confessor
            - 4 Graceful Calligrapher
            - 1 Ground Crew
            - 4 Hammer of Might
            - 4 Inquisitor's Blade
            - 2 Ironclad Oath
            - 1 Ironclad Oath
            - 1 Lawman's Sidearm
            - 4 Mantle of Justice
            - 4 Peacekeeper's Prod
[ 44/ 99] Processing file ss042.png...
            - 4 Rabblerouser
            - 1 Rakano Sheriff
            - 4 Shard of the Spire
            - 2 Sheriff Marley
            - 1 Sheriff's Hat
            - 4 Shielded Shortbarrel
            - 4 Silverwing Avenger
            - 2 Sky Crew
            - 4 Spiked Buckler
            - 1 Spiritblade Stalker
            - 4 Stalwart Shie

            - 4 Iceknuckle Jotun
            - 4 Lumbering Gruan
            - 1 Moonlight Huntress
            - 4 North-Wind Herald
            - 2 Sapphire Dragon
            - 2 Scouting Party
            - 3 Shamanic Blast
            - 4 Slope Sergeant
[ 63/ 99] Processing file ss061.png...
            - 2 Speardiver
            - 3 Thunderstrike Dragon
            - 4 Adaptive Predator
            - 4 Araktodon
            - 4 Bellowing Thunderfoot
            - 4 Elysian Trailblazer
            - 2 Enraged Araktodon
            - 4 Hooru Teachings
            - 4 Icequake
            - 2 Mistveil Drake
            - 1 Novice Herdrider
            - 2 Rimescale Draconus
[ 64/ 99] Processing file ss062.png...
            - 1 Eilyn, Queen of the Wilds
            - 3 Surveying Mantasaur
            - 3 Channel the Tempest
            - 1 Shadow Sigil
            - 4 Amethyst Monument
            - 4 Amethyst Waystone
            - 4 Affliction
            - 1 Arachnophobia
       

            - 1 Ijin's Choice
            - 2 Rakano Artisan
            - 1 Rakano Artisan
            - 4 Crownwatch Deserter
            - 4 Sower of Dissent
            - 4 Sword of Icaria
            - 1 Whirling Duo
            - 4 Battleblur Centaur
            - 4 Longhorn Sergeant
            - 1 Longhorn Sergeant
[ 83/ 99] Processing file ss081.png...
            - 4 Renegade Valkyrie
            - 1 Renegade Valkyrie
            - 3 Righteous Fury
            - 1 Rise to the Challenge
            - 4 Field Captain
            - 1 Field Captain
            - 4 Jekk, the Bounty Hunter
            - 1 Starsteel Daisho
            - 4 Combrei Banner
            - 4 Seat of Progress
            - 4 Awakened Student
            - 4 Desert Marshal
[ 84/ 99] Processing file ss082.png...
            - 4 Jaril, Amaran Ghostblade
            - 4 Safe Return
            - 1 Safe Return
            - 4 Combrei Emissary
            - 4 Combrei Healer
            - 1 Combrei Healer
       

In [53]:
my_collection=[]
for card in library:
    if library[card]['owned']>0:
        my_collection.append('{} {} (Set{} #{})'.format(library[card]['owned'],
                                                        card,
                                                        library[card]['set'],
                                                        library[card]['id']))
with open(collection_file,'w') as f:
    f.writelines([card+'\n' for card in my_collection])

Errors are to be expected at the last few cards, where there is actually no card on the screenshot.

In [54]:
errors

['[ERROR] Problem identifying card 11 in screenshot ss097.png!',
 '[ERROR] Problem identifying card 12 in screenshot ss097.png!']